In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import attila_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
fpath = '../../resources/tunahan/Mathys_SC_iMAT/SC_astrocyte_iMAT_binary_models_10.04.22.txt'
imat = pd.read_csv(fpath, header=None).values

In [3]:
fpath = '../../resources/tunahan/Mathys_SC_iMAT/SC_astrocyte_metadata_10.04.22.xlsx'
df = pd.read_excel(fpath, dtype=pd.Categorical)

In [4]:
def read_imat(celltype='astrocyte'):
    imatpath = '../../resources/tunahan/Mathys_SC_iMAT/SC_' + celltype + '_iMAT_binary_models_10.04.22.txt'
    metapath = '../../resources/tunahan/Mathys_SC_iMAT/SC_' + celltype + '_metadata_10.04.22.xlsx'
    meta = pd.read_excel(metapath, dtype='category')
    meta = meta.rename(dict(zip(meta.index, meta.SampleID)))
    meta['Subject'] = meta.SampleID.str.split('.').apply(lambda x: x[1])
    imat = pd.read_csv(imatpath, header=None).values.transpose()
    imat = pd.DataFrame(imat, index=meta.SampleID)
    val = pd.concat([meta.drop(['ProjID', 'SampleID'], axis=1), imat], axis=1)
    return(val)

celltypes = 'astrocyte', 'exneuron', 'microglia'
imat = {c: read_imat(c) for c in celltypes}

AttributeError: 'DataFrame' object has no attribute 'SampleID'

In [ ]:
%connect_info